## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)
#print(mouse_metadata)
#print(study_results)

# Combine the data into a single dataset
mice_study = pd.merge(study_results,mouse_metadata, on="Mouse ID")

# Display the data table for preview
mice_study.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [2]:
# Checking the number of mice.
mouse_count=mice_study["Mouse ID"].value_counts().count()
mouse_count

249

In [3]:
# duplicate mouse by ID and Timepoint
duplicate_mouse_df= mice_study[["Mouse ID", "Timepoint"]]
duplicate_df = duplicate_mouse_df[duplicate_mouse_df.duplicated(keep ="last")]
duplicate_df
#unique Mouse ID as a duplicate found
mouseID_duplicate= duplicate_df[["Mouse ID"]].squeeze().unique()
mouseID_duplicate


array(['g989'], dtype=object)

In [4]:
# Get all the data for the duplicate mouse ID
all_data_duplicate = mice_study[mice_study["Mouse ID"].isin(mouseID_duplicate)]
all_data_duplicate

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
860,g989,0,45.000000,0,Propriva,Female,21,26
861,g989,0,45.000000,0,Propriva,Female,21,26
862,g989,5,48.786801,0,Propriva,Female,21,26
863,g989,5,47.570392,0,Propriva,Female,21,26
864,g989,10,51.745156,0,Propriva,Female,21,26
865,g989,10,49.880528,0,Propriva,Female,21,26
866,g989,15,51.325852,1,Propriva,Female,21,26
867,g989,15,53.442020,0,Propriva,Female,21,26
868,g989,20,55.326122,1,Propriva,Female,21,26
869,g989,20,54.657650,1,Propriva,Female,21,26


In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_mouse_df = mice_study.set_index(["Mouse ID"])
indexed_mouse_ID = clean_mouse_df.drop(mouseID_duplicate)
indexed_mouse_ID.sort_values(by="Timepoint").head()

,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
Mouse ID,,,,,,,
b128,0,45.0,0,Capomulin,Female,9,22
v409,0,45.0,0,Placebo,Female,16,25
u946,0,45.0,0,Propriva,Male,5,30
w140,0,45.0,0,Zoniferol,Female,19,30
a577,0,45.0,0,Infubinol,Female,6,25


In [6]:
# Checking the number of mice in the clean DataFrame.
grouped_indexed_ID=indexed_mouse_ID.groupby(["Mouse ID"])
mouse_clean_count=len(grouped_indexed_ID.count())
mouse_clean_count

248

## Summary Statistics

In [7]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
#reseting index to get a table
reset_data = indexed_mouse_ID.reset_index() 
reset_data  




,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22
...,...,...,...,...,...,...,...,...
1875,m601,25,33.118756,1,Capomulin,Male,22,17
1876,m601,30,31.758275,1,Capomulin,Male,22,17
1877,m601,35,30.834357,1,Capomulin,Male,22,17
1878,m601,40,31.378045,1,Capomulin,Male,22,17


In [10]:
#extracting columns Drug Regimen and Tumor Volume(mm3)
drugs_regimen_df=reset_data[["Drug Regimen","Tumor Volume (mm3)"]]
drugs_regimen_df

,Drug Regimen,Tumor Volume (mm3)
0,Capomulin,45.000000
1,Capomulin,45.651331
2,Capomulin,43.270852
3,Capomulin,43.784893
4,Capomulin,42.731552
...,...,...
1875,Capomulin,33.118756
1876,Capomulin,31.758275
1877,Capomulin,30.834357
1878,Capomulin,31.378045


In [11]:
#Calculating:
#mean
drugs_mean = drugs_regimen_df.groupby("Drug Regimen").mean()
drugs_mean = drugs_mean.rename(columns={"Tumor Volume (mm3)" : "Mean Tumor Volume"})
drugs_mean


,Mean Tumor Volume
Drug Regimen,
Capomulin,40.675741
Ceftamin,52.591172
Infubinol,52.884795
Ketapril,55.235638
Naftisol,54.331565
Placebo,54.033581
Propriva,52.320930
Ramicane,40.216745
Stelasyn,54.233149


In [12]:
#median
drugs_median = drugs_regimen_df.groupby("Drug Regimen").median()
drugs_median = drugs_median.rename(columns={"Tumor Volume (mm3)" : "Median Tumor Volume"})
drugs_median

,Median Tumor Volume
Drug Regimen,
Capomulin,41.557809
Ceftamin,51.776157
Infubinol,51.820584
Ketapril,53.698743
Naftisol,52.509285
Placebo,52.288934
Propriva,50.446266
Ramicane,40.673236
Stelasyn,52.431737


In [13]:
#variance
drugs_variance = drugs_regimen_df.groupby("Drug Regimen").var()
drugs_variance = drugs_variance.rename(columns={"Tumor Volume (mm3)" : "Tumor Volume Variance"})
drugs_variance

,Tumor Volume Variance
Drug Regimen,
Capomulin,24.947764
Ceftamin,39.290177
Infubinol,43.128684
Ketapril,68.553577
Naftisol,66.173479
Placebo,61.168083
Propriva,43.852013
Ramicane,23.486704
Stelasyn,59.450562


In [14]:
#standard deviation
drugs_std =drugs_regimen_df.groupby("Drug Regimen").std()
drugs_std = drugs_std.rename(columns={"Tumor Volume (mm3)" : "Tumor Volume Stand.Dev."})
drugs_std

,Tumor Volume Stand.Dev.
Drug Regimen,
Capomulin,4.994774
Ceftamin,6.268188
Infubinol,6.567243
Ketapril,8.279709
Naftisol,8.134708
Placebo,7.821003
Propriva,6.622085
Ramicane,4.846308
Stelasyn,7.710419


In [15]:
#SEM
drugs_sem = drugs_regimen_df.groupby("Drug Regimen").sem()
drugs_sem  = drugs_sem.rename(columns={"Tumor Volume (mm3)" : "Tumor Volume Stand.Error"})
drugs_sem

,Tumor Volume Stand.Error
Drug Regimen,
Capomulin,0.329346
Ceftamin,0.469821
Infubinol,0.492236
Ketapril,0.603860
Naftisol,0.596466
Placebo,0.581331
Propriva,0.544332
Ramicane,0.320955
Stelasyn,0.573111


In [16]:
# Assemble the resulting series into a single summary dataframe.
summary_drug_stats_df = pd.merge(drugs_mean, drugs_median, on="Drug Regimen" )
summary_drug_stats1_df = pd.merge(drugs_variance,drugs_std, on ="Drug Regimen")
summary_drug_stats2_df = pd.merge(summary_drug_stats1_df, drugs_sem, on ="Drug Regimen")
final_summary_stats = pd.merge(summary_drug_stats_df, summary_drug_stats2_df, on ="Drug Regimen")
final_summary_stats

,Mean Tumor Volume,Median Tumor Volume,Tumor Volume Variance,Tumor Volume Stand.Dev.,Tumor Volume Stand.Error
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [17]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line

stats_agg = drugs_regimen_df.groupby("Drug Regimen").agg(["mean", "median", "var", "std", "sem"])
stats_agg

Tumor Volume (mm3)                                          
                           mean     median        var       std       sem
Drug Regimen                                                             
Capomulin             40.675741  41.557809  24.947764  4.994774  0.329346
Ceftamin              52.591172  51.776157  39.290177  6.268188  0.469821
Infubinol             52.884795  51.820584  43.128684  6.567243  0.492236
Ketapril              55.235638  53.698743  68.553577  8.279709  0.603860
Naftisol              54.331565  52.509285  66.173479  8.134708  0.596466
Placebo               54.033581  52.288934  61.168083  7.821003  0.581331
Propriva              52.320930  50.446266  43.852013  6.622085  0.544332
Ramicane              40.216745  40.673236  23.486704  4.846308  0.320955
Stelasyn              54.233149  52.431737  59.450562  7.710419  0.573111
Zoniferol             53.236507  51.818479  48.533355  6.966589  0.516398

## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using Pandas.



In [10]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
